# Enunciado del problema 1

En el archivo auto.csv se encuentran los siguientes datos de diferentes automóviles:

Cilindros
Cilindrada
Potencia
Peso
Aceleración
Año del coche
Origen
Consumo (mpg)
Las unidades de las características de los automóviles no se encuentran en el sistema internacional. La variable “origen” es un código que identifica al país de origen.

Crea un modelo con él para que se pueda estimar el consumo de un vehículo a partir del resto de las variables.

In [1]:
%pylab
%matplotlib inline

%config InlineBackend.figure_format = 'retina'

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


## Imports

Importamos los módulos necesarios para resolver el ejercicio

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import  median_absolute_error

from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import VarianceThreshold

## Carga

Carga del csv 'auto.csv' en un dataFrame y visualización de las 5 primeras líneas para ver la estrucutura.

In [3]:
df = pd.read_csv('auto.csv')
df.head()

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,mpg
0,8,307.0,130.0,3504.0,12.0,70,1,18.0
1,8,350.0,165.0,3693.0,11.5,70,1,15.0
2,8,318.0,150.0,3436.0,11.0,70,1,18.0
3,8,304.0,150.0,3433.0,12.0,70,1,16.0
4,8,302.0,140.0,3449.0,10.5,70,1,17.0


Visualizamos los tipos de variables y la distribución de los datos faltantes que contiene el dataFrame, también vemos el tamaño del DataFrame.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392 entries, 0 to 391
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cylinders     392 non-null    int64  
 1   displacement  392 non-null    float64
 2   horsepower    392 non-null    float64
 3   weight        392 non-null    float64
 4   acceleration  392 non-null    float64
 5   model_year    392 non-null    int64  
 6   origin        392 non-null    int64  
 7   mpg           392 non-null    float64
dtypes: float64(5), int64(3)
memory usage: 24.6 KB


## Preparación del dataset para entrenamiento

Separamos la data (x) del target (y)

In [5]:
#data
x = df[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin']]

#target
y = df.mpg

Normalización de las variables en diferentes unidades para evitar sesgos.

In [6]:
columns = ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin']

scaler = StandardScaler().fit(x)
data = scaler.transform(x)
    
x = pd.DataFrame(data = data, columns = columns)

Visualizamos las 5 primeras líneas del dataFrame con las variables normalizadas.

In [7]:
x.head()

,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
0,1.483947,1.077290,0.664133,0.620540,-1.285258,-1.625315,-0.716641
1,1.483947,1.488732,1.574594,0.843334,-1.466724,-1.625315,-0.716641
2,1.483947,1.182542,1.184397,0.540382,-1.648189,-1.625315,-0.716641
3,1.483947,1.048584,1.184397,0.536845,-1.285258,-1.625315,-0.716641
4,1.483947,1.029447,0.924265,0.555706,-1.829655,-1.625315,-0.716641


Seleccionamos las variables que tienen suficiente varianza como para explicar la variable objetivo.

In [8]:
var_th = VarianceThreshold(threshold = 0.60)
x_var = var_th.fit_transform(x)

print("Variables originales ", x.shape[1])
print("Variables finales ", x_var.shape[1])

print("Listado de variables ", np.asarray(list(x))[var_th.get_support()])

Variables originales  7
Variables finales  7
Listado de variables  ['cylinders' 'displacement' 'horsepower' 'weight' 'acceleration'
 'model_year' 'origin']


## Train y test

In [9]:
#separamos train y test fijando la semilla para reproducibilidad de datos
x_train, x_test, y_train, y_test = train_test_split(x_var, y, random_state = 1)

## Creación del modelo

Como disponemos de la variable que queremos predecir, debemos utilizar un modelo supervisado, al ser la variable objetivo una variable continua optamos por una **regresión lineal** como modelo.

In [10]:
#creamos el modelo
model = LinearRegression()

#entrenamos el modelo
model.fit(x_train, y_train)

#predicciones
predict_train = model.predict(x_train)
predict_test = model.predict(x_test)

Evaluamos el R2 del modelo.

In [11]:
#evaluación R2
print('R2 en train: ', model.score(x_train, y_train))
print('R2 en test: ', model.score(x_test, y_test))

R2 en train:  0.8153339627635713
R2 en test:  0.8295700536937243


Los resultados son bastante buenos tanto en train como test, lo que parece indicar que no se ha producido overfitting y que el modelo hace buenas predicciones.
Vemos cuáles son los coeficientes del modelo.

In [12]:
#coeficientes modelo
print('Los coeficientes del modelo son: ', model.coef_)

Los coeficientes del modelo son:  [-0.34496923  1.86818123 -0.25651678 -5.87993156  0.5921522   2.75396901
  1.04584578]
